![header](../figures/logos_partenaires_DC_WOC-ESA.jpg)

# Mediterranean: Benchmark DUACS plot maps
 

***
**Authors:** Datlas <br>
**Copyright:** 2023 Datlas <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h1><center> Mediterranean: Benchmark of DUACS plot maps</center></h1>
<h4><center> The notebook aims to plot maps produced by the DUACS system in the Mediterranean region. </center></h4> 
</div>

In [6]:
from glob import glob
import numpy as np
import os

In [7]:
import sys
sys.path.append('..')
from src.mod_plot import *
from src.mod_stat import *
from src.mod_spectral import *
from src.mod_interp import *
from src.mod_switchvar import *

In [8]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

<div class="alert alert-info" role="alert">

<h2>0. Parameters</h2>

</div>

In [9]:
time_min = '2019-01-15'                                        # time min for analysis
time_max = '2019-12-15'                                        # time max for analysis 

# Region details
region = 'Mediterranean'
lon_min = -5                                          # domain min longitude
lon_max = 25                                          # domain max longitude
lat_min = 35.                                          # domain min latitude
lat_max = 47.                                          # domain max latitude


box_lonlat = {'lon_min':lon_min,'lon_max':lon_max,'lat_min':lat_min,'lat_max':lat_max}

method_name = 'DUACS'


output_dir = '../results/'+method_name+'/'                                      # output directory path
os.system(f'mkdir -p {output_dir}')

figures_dir = '../figures/'+method_name+'/'                                      # figures directory path
os.system(f'mkdir -p {figures_dir}')

0

<div class="alert alert-info" role="alert">

<h2>1. Input files</h2>

</div>

## Sea Surface Height maps to evaluate

In [10]:
%%time
list_of_maps = sorted(glob('../data/maps/DUACS_'+region+'.nc'))
ds_maps = xr.open_mfdataset(list_of_maps, combine='nested', concat_dim='time')
ds_maps = ds_maps.sel(time=slice(time_min, time_max))
ds_maps

CPU times: user 57.7 ms, sys: 90.1 ms, total: 148 ms
Wall time: 311 ms


<xarray.Dataset>
Dimensions:    (latitude: 52, longitude: 124, time: 335)
Coordinates:
  * latitude   (latitude) float32 34.62 34.88 35.12 35.38 ... 46.88 47.12 47.38
  * longitude  (longitude) float64 -5.375 -5.125 -4.875 ... 24.88 25.12 25.38
  * time       (time) datetime64[ns] 2019-01-15 2019-01-16 ... 2019-12-15
Data variables:
    sla        (time, latitude, longitude) float64 dask.array<chunksize=(335, 52, 124), meta=np.ndarray>
    ugosa      (time, latitude, longitude) float64 dask.array<chunksize=(335, 52, 124), meta=np.ndarray>
    vgosa      (time, latitude, longitude) float64 dask.array<chunksize=(335, 52, 124), meta=np.ndarray>
    adt        (time, latitude, longitude) float64 dask.array<chunksize=(335, 52, 124), meta=np.ndarray>
    ugos       (time, latitude, longitude) float64 dask.array<chunksize=(335, 52, 124), meta=np.ndarray>
    vgos       (time, latitude, longitude) float64 dask.array<chunksize=(335, 52, 124), meta=np.ndarray>

## Plot DUACS geostrophic currents movie

In [11]:
movie(ds_maps,name_var='uv', method=method_name, region=region, dir_output='../results/',dim_name=['time','latitude','longitude'],
          framerate=24,Display=True,clim=[0,1.2],cmap='YlGnBu_r', newmovie=True, figsize=(12,6))


## Compute and plot DUACS relative vorticity movie

In [12]:
rv = currents_to_relative_vorticity(np.array(ds_maps.ugos),np.array(ds_maps.vgos), ds_maps.longitude,ds_maps.latitude)

ds_maps = ds_maps.assign(rv=lambda ds_maps: ds_maps.sla * 0 + rv)

In [13]:
movie(ds_maps,name_var='rv', method=method_name, region=region, dir_output='../results/',dim_name=['time','latitude','longitude'],
          framerate=24,Display=True,clim=[-0.8,0.8],cmap='RdBu_r', newmovie=True, figsize=(12,6))
